In [1]:
from matplotlib import pyplot as plt
import warnings
import keras
from keras.callbacks import EarlyStopping
from keras.regularizers import l2 
from keras.models import Sequential
from keras.layers import Reshape
from keras.layers import Dense, Dropout, Activation, Flatten,Conv1D, GlobalAveragePooling1D, MaxPooling1D,AveragePooling1D
from keras.layers import Conv2D, MaxPooling2D,Conv1D,GlobalMaxPooling1D,MaxPooling1D,average, concatenate,RepeatVector,Lambda,add,subtract,Activation
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Input, Model
from sklearn import metrics as mt
from skimage.io import imshow
from sklearn import preprocessing
from matplotlib.patches import Rectangle
from keras import backend as K
from keras.layers.normalization import BatchNormalization
warnings.filterwarnings('ignore')
import featureHelper as fh
import labelHelper as lh

C:\Users\razer\AppData\Local\Continuum\anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
C:\Users\razer\AppData\Local\Continuum\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\razer\AppData\Local\Continuum\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])

In [2]:
ll=lh.Label('..//2019_fall_labels.csv')
folder = "C:\\2019_fall_video_features"

video_clips_length=30
time_lag=2
move_threshold=150
stride=video_clips_length

In [3]:
train_video,train_video_label,test_video,test_video_label = fh.make_train_test_data(folder,ll,video_clips_length,time_lag,move_threshold,stride)

before deop we have 84 videos
Video 924 did not find
we have 83 video after drop


In [4]:
print(train_video.shape)
print(train_video_label.shape)
print(test_video.shape)
print(test_video_label.shape)

(2026, 30, 28)
(2026, 6)
(676, 30, 28)
(676, 6)
